In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import numpy as np

In [2]:
#Loading and Preparing the MNIST Data Set
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
#print(x_train)
# print(y_train)
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
#Flatten images to 1-D vector of 784 features (28*28).
num_features=784
print(x_train.shape)
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
print(x_train.shape)
# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.
# print(x_train)

(60000, 28, 28)
(60000, 784)


In [3]:
#Setting Up Hyperparameters and Data Set Parameters
#MNIST dataset parameters
num_classes = 10 # 0 to 9 digits
num_features = 784 # 28*28

# Training parameters
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

In [4]:
#SHUFFLING AND BATCHING THE DATA
#Use tf.data API to shuffle and batch data.

train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)
print(train_data)

<PrefetchDataset shapes: ((None, 784), (None,)), types: (tf.float32, tf.uint8)>


In [5]:
#initialize weight and biases
W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")
print(W)
b = tf.Variable(tf.zeros([num_classes]), name="bias")
print(b)

<tf.Variable 'weight:0' shape=(784, 10) dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>
<tf.Variable 'bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>


In [6]:
#define logistic regression and cost function
def logistic_regression(x):
    sm=tf.nn.softmax(tf.matmul(x, W) + b)
    return sm
#loss function.

def cross_entropy(y_pred, y_true):
    #label to a one hot vector
    y_true = tf.one_hot(y_true, depth=num_classes)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    #Calculate cross-entropy.
    entropy=tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))
    return entropy

In [7]:
def accuracy(y_pred, y_true):
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    acc=tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return acc
optimizer = tf.optimizers.SGD(learning_rate)

In [8]:
def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)
    #calculate gradient
    gradients = g.gradient(loss, [W, b])
    # Update W and b
    optimizer.apply_gradients(zip(gradients, [W, b]))

In [9]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    #update W and b values
    run_optimization(batch_x, batch_y)
    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 997.842285, accuracy: 0.683594
step: 100, loss: 1029.174927, accuracy: 0.765625
step: 150, loss: 1595.479004, accuracy: 0.652344
step: 200, loss: 1280.970703, accuracy: 0.585938
step: 250, loss: 633.649353, accuracy: 0.792969
step: 300, loss: 430.701538, accuracy: 0.878906
step: 350, loss: 633.924133, accuracy: 0.832031
step: 400, loss: 462.249725, accuracy: 0.867188
step: 450, loss: 798.943237, accuracy: 0.808594
step: 500, loss: 582.839722, accuracy: 0.824219
step: 550, loss: 656.069458, accuracy: 0.832031
step: 600, loss: 602.234497, accuracy: 0.777344
step: 650, loss: 642.905518, accuracy: 0.777344
step: 700, loss: 762.010437, accuracy: 0.781250
step: 750, loss: 689.004456, accuracy: 0.812500
step: 800, loss: 1117.997559, accuracy: 0.703125
step: 850, loss: 622.165833, accuracy: 0.796875
step: 900, loss: 592.928955, accuracy: 0.812500
step: 950, loss: 437.539001, accuracy: 0.859375
step: 1000, loss: 685.601807, accuracy: 0.792969


In [10]:
#Testing model
pred = logistic_regression(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.812900
